# ETL Documentation

### Introduction

The data is scraped from Agoda once a day for seven days from 13.11.2021 to 19.11.2021.

**Search Criteria:**\
  **- 2 Adults**\
  **- one Bedroom**\
  **- in USD**\
  **- 01.12.2021 ~ 02.12.2021 (1 night)**

### ETL Steps 

**Section 1: Data Extraction**

**Section 2: Data Transformation**

**Section 3: Data Loading**

 ### SECTION 1: Data Extraction

Step 1: import all the packages needed

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import ElementNotVisibleException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
import time
import csv
import datetime

Step 2: set up the driver to open the url

In [ ]:
s=Service("C:\Program Files (x86)\chromedriver.exe")

driver = webdriver.Chrome(service=s)

driver.get("https://www.agoda.com/en-gb/search?guid=95021892-f462-4acc-9740-e9f344ba0993&\
            asq=N9jn0hTf%2Fx2lkUgOHefX0Jufa9Vwpz6XltTHq4n%2B9gOSH6DsqyDiss6fR9e7cCJtdYyNUYml1sylYahk8CBCc\
            GWFZ4kyb4zbzY4f4c3DpymuDW2eVSKynxmmSvy%2B8%2F%2B7Bb%2BZ58XrzTZovriM0y21pk27RO0Ag1B%2F1Nu%2FTc4\
            q82aamP2nLmh6t2KBvIKH318BifxUXnPHGcx2NEGs0tAtdA%3D%3D&city=17193&tick=637722838468&locale=en-gb\
            &ckuid=93b760bf-2ce1-44d7-9a19-64211e9c03bf&prid=0&gclid=CjwKCAjwz5iMBhAEEiwAMEAwGEXGUhwNVAn6nKE\
            yUjLXTC00A4H70ohLultZnP_6n1uTD8f2dTL9DxoCrl8QAvD_BwE%26amp%3Bamp%3B%23164%3Bcy&currency=USD&corre\
            lationId=1d669865-810a-4bea-a307-0f1ac2c694da&pageTypeId=103&realLanguageId=16&languageId=1&origin\
            =CH&cid=1891458&tag=24bd4b3f-b6a1-50d5-e639-d9c49ca41c49&userId=93b760bf-2ce1-44d7-9a19-64211e9c\
            03bf&whitelabelid=1&loginLvl=0&storefrontId=3&currencyId=7&currencyCode=USD&htmlLanguage=en-gb&\
            cultureInfoName=en-gb&machineName=am-crweb-4019&trafficGroupId=5&sessionId=v3szev4vkyeu1zmk3nm55\
            xqa&trafficSubGroupId=122&aid=82361&useFullPageLogin=true&cttp=4&isRealUser=true&mode=production&\
            checkIn=2021-12-01&checkOut=2021-12-02&rooms=1&adults=2&children=0&priceCur=USD&los=1&textToSearch=\
            Bali&productType=-1&travellerType=1&familyMode=off")

wait_time = 3

Step 3: define a function to scroll through the dynamic webpage

In [ ]:
def scrolling(driver):
    
    # Get page height.
    previous_height = driver.execute_script("return document.body.scrollHeight")

    while True:

        # Scroll to the bottom.
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load the page.
        time.sleep(wait_time)

        # Calculate new scroll height and compare with last scroll height.
        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == previous_height:

            break

        previous_height = new_height

Step 4: Scrape data needed

In [ ]:
#initialize a list for storing the scraped data        
cards_for_parsing = []


#use a while loop to go through all pages
while True:

    print("length of cards_for_parsing: " + str(len(cards_for_parsing)))    

    scrolling(driver)
    time.sleep(wait_time)
    
    #find all the cards which contain the information needed
    hotel_cards = driver.find_elements_by_class_name("PropertyCard__Container") 

    #for loop to go through all the cards and get the details of hotels
    for card in hotel_cards:

        name = card.find_element_by_xpath('.//h3[@data-selenium="hotel-name"]').text

        address = card.find_element_by_xpath('.//span[@data-selenium="area-city-text"]').text

        hotel_stars = card.find_element_by_class_name('star-rating')
        star = hotel_stars.find_elements_by_tag_name('i')

        if len(star)>0:
            star = star[0].get_attribute("title")
        else:
            star = "-"

        try: #for hotels without certain information, "N.A" is added 
            sale_price = card.find_element_by_class_name("PropertyCardPrice__Value").text
        except NoSuchElementException:
            sale_price = "N.A"
        try:
            rating = card.find_element_by_class_name("sc-dtTInj.kqQFCk.kite-js-Typography").text
        except NoSuchElementException:
                rating = "N.A"
        try:
            number_of_reviews = card.find_element_by_class_name("sc-kEjbxe.ekqPym.sc-gsTCUz.cHVkfi").text
        except NoSuchElementException:
            number_of_reviews = "N.A"
        
        #store the data in a dictionary and append it to the list
        cards_for_parsing.append({
            "name" : name,
            "price" : sale_price,
            "rating" : rating,
            "number_of_reviews": number_of_reviews,
            "address": address,
            "stars": star
                })
        
    try: #try to look for the next page button and click on it
        next_button = driver.find_element_by_class_name("pagination2__next")
        driver.execute_script("arguments[0].click();", next_button)
        time.sleep(wait_time)
    except NoSuchElementException: #if the button is not found, last page is reached
        print("Done Crwaling")
        break

Step 5: export the scraped data

In [ ]:
today = datetime.date.today().strftime("%d-%m-%Y") #get the date so that it can be added to the file name
keys = cards_for_parsing[0].keys() #get the keys for column names

#write the data into csv file 
with open('Agoda -' + today + '.csv' , 'w', newline='', encoding='utf-8') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(cards_for_parsing)

### SECTION 2: Data Tranformation

#### Step 1 Clean Impurities

Impurities:

   1.  N.A values
   2. Number of reviews column contains string
   3. Inconsistent unit of distance (m and km)
   4. Hotel names consist of unwanted characters e.g.✰ Camaya Bali Butterfly - Magical Bamboo House ✰
   5. 0 star is shown as "star"
   6. duplicated rows



Step 1.1 - import packages needed

In [5]:
import glob
import re
import pandas as pd 
import numpy as np

Step 1.2 - read all the scraped data into a dataframe

In [2]:
path = r'C:\Users\agnes\switchdrive\SyncVM\CIP H21\Project\Source Data'
all_files = glob.glob(path + "/*.csv")

#create an empty list for storing all the csv files 
li = []

for filename in all_files:
    dataframe = pd.read_csv(filename, index_col=None, header=0)
    dataframe['date'] = filename[-18:-8]
    li.append(dataframe)

#combine all the dataframes into one dataframe
df = pd.concat(li, axis=0, ignore_index=True)


In [3]:
df.head()

,name,price,rating,number_of_reviews,address,stars,date
0,Amnaya Resort Kuta,21,9.2,"8,352 reviews","Kuta, Bali - 3.4 km to center",4 star,13-11-2021
1,The Kayon Jungle Resort,117,9.4,"1,162 reviews","Ubud, Bali - 30.1 km to center",5 star,13-11-2021
2,Astagina Resort Villa and Spa,21,8.5,"2,327 reviews","Seminyak, Bali - 99 m to center",4 star,13-11-2021
3,Mulia Resort,161,8.9,372 reviews,"Nusa Dua, Bali - 13.8 km to center",5 star,13-11-2021
4,Mahanidhi Luxury Private Villa #4,39,N.A,N.A,"Ubud, Bali - 22.2 km to center",4 star,13-11-2021


Step 1.3 - Clean impurities 

In [4]:
##Remove Impurity - N.A values in price 
df = df[df.price != 'N.A']

#convert price and rating to numeric
df[['price', 'rating']] = df[['price', 'rating']].apply(pd.to_numeric, errors='coerce', axis=1)

##Remove Impurity - duplicated rows
df = df.drop_duplicates()

In [5]:
##Remove Impurity - number_of_reviews column shouldn't include string 

#extract only the number
df['number_of_reviews'] = df['number_of_reviews'].str.extract('(\d.*)reviews', expand=True)

#remove commas
df['number_of_reviews'] = df['number_of_reviews'].str.replace(',', '')

#convert number_of_reviews to numeric
df["number_of_reviews"] = pd.to_numeric(df["number_of_reviews"], errors='coerce')

In [6]:
##Remove Impurity - inconsistent unit of distance 

#create a temparary column with distance to center 
df['temp_col'] = df['address'].str.extract('(\d.*)to', expand=True)

#split the temp_col into two columns - one with distance, one with unit
df[['distance','unit']] = df.temp_col.str.split(expand=True)

#change the data type of distance column to numeric
df["distance"] = pd.to_numeric(df["distance"],errors='coerce')

#convert values in meter to kilometer
df.loc[df.unit == 'm', 'distance'] = round(df.distance*0.001, 1) 

In [7]:
##Remove Impurities in stars column

#the empty star should be 0 star
df.loc[df.stars == ' star', 'stars'] = "0 star"

#only keep the number in the column
df['stars'] = df['stars'].str.replace('star', '')

#change this column to numeric
df["stars"] = pd.to_numeric(df["stars"])

In [8]:
##Remove Impurities - unwanted special characters in name column
df["name"]=df["name"].replace('[^A-Za-z0-9 ]+','',regex=True)

In [9]:
df.head()

,name,price,rating,number_of_reviews,address,stars,date,temp_col,distance,unit
0,Amnaya Resort Kuta,21.0,9.2,8352.0,"Kuta, Bali - 3.4 km to center",4.0,13-11-2021,3.4 km,3.4,km
1,The Kayon Jungle Resort,117.0,9.4,1162.0,"Ubud, Bali - 30.1 km to center",5.0,13-11-2021,30.1 km,30.1,km
2,Astagina Resort Villa and Spa,21.0,8.5,2327.0,"Seminyak, Bali - 99 m to center",4.0,13-11-2021,99 m,0.1,m
3,Mulia Resort,161.0,8.9,372.0,"Nusa Dua, Bali - 13.8 km to center",5.0,13-11-2021,13.8 km,13.8,km
4,Mahanidhi Luxury Private Villa 4,39.0,NaN,NaN,"Ubud, Bali - 22.2 km to center",4.0,13-11-2021,22.2 km,22.2,km


#### Step 2 - Data Enrichment

Step 2.1 - based on the distance create a new column with catergorical variables

In [10]:
# create a list of conditions
conditions = [(df['distance'].isnull()),
    (df['distance'] <= 1),
    (df['distance'] > 1) & (df['distance'] <= 2.5),
    (df['distance'] > 2.5) & (df['distance'] <= 5),
    (df['distance'] > 5)
    ]

# create a list of the values we want to assign for each condition
values = ['in city center','very close', 'close', 'far', 'very far']

# create a new column and use np.select to assign values to it using our lists as arguments
df['distance_from_city_center'] = np.select(conditions, values)

#drop unnecessary columns
df = df.drop(['temp_col', 'distance', 'unit'], axis=1)

In [11]:
df.head()

,name,price,rating,number_of_reviews,address,stars,date,distance_from_city_center
0,Amnaya Resort Kuta,21.0,9.2,8352.0,"Kuta, Bali - 3.4 km to center",4.0,13-11-2021,far
1,The Kayon Jungle Resort,117.0,9.4,1162.0,"Ubud, Bali - 30.1 km to center",5.0,13-11-2021,very far
2,Astagina Resort Villa and Spa,21.0,8.5,2327.0,"Seminyak, Bali - 99 m to center",4.0,13-11-2021,very close
3,Mulia Resort,161.0,8.9,372.0,"Nusa Dua, Bali - 13.8 km to center",5.0,13-11-2021,very far
4,Mahanidhi Luxury Private Villa 4,39.0,NaN,NaN,"Ubud, Bali - 22.2 km to center",4.0,13-11-2021,very far


Step 2.2 - split address and create new columns region and country

In [12]:
df['city'] = df['address'].str.split(',').str[0]
df['region'] = 'Bali'
df['country'] = 'Indonesia'

In [13]:
df.head()

,name,price,rating,number_of_reviews,address,stars,date,distance_from_city_center,city,region,country
0,Amnaya Resort Kuta,21.0,9.2,8352.0,"Kuta, Bali - 3.4 km to center",4.0,13-11-2021,far,Kuta,Bali,Indonesia
1,The Kayon Jungle Resort,117.0,9.4,1162.0,"Ubud, Bali - 30.1 km to center",5.0,13-11-2021,very far,Ubud,Bali,Indonesia
2,Astagina Resort Villa and Spa,21.0,8.5,2327.0,"Seminyak, Bali - 99 m to center",4.0,13-11-2021,very close,Seminyak,Bali,Indonesia
3,Mulia Resort,161.0,8.9,372.0,"Nusa Dua, Bali - 13.8 km to center",5.0,13-11-2021,very far,Nusa Dua,Bali,Indonesia
4,Mahanidhi Luxury Private Villa 4,39.0,NaN,NaN,"Ubud, Bali - 22.2 km to center",4.0,13-11-2021,very far,Ubud,Bali,Indonesia


#### Step 3 - Formate the data to be consistent with other students

In [14]:
#keep only the columns needed for answering group questions
col_selection = ['date', 'name', 'rating', 'price', 'number_of_reviews', 'stars', 'city','country','region']

#rename the columns to be consistent with other students
df = df[col_selection].drop_duplicates().rename(columns={"number_of_reviews":'review', 'stars': 'star', 'rating':'rate', 'name':'title'})

#format the date to be consistent with other students
df['date'] = pd.to_datetime(df['date']).dt.strftime('%Y-%m-%d')

In [15]:
df.head()

,date,title,rate,price,review,star,city,country,region
0,2021-11-13,Amnaya Resort Kuta,9.2,21.0,8352.0,4.0,Kuta,Indonesia,Bali
1,2021-11-13,The Kayon Jungle Resort,9.4,117.0,1162.0,5.0,Ubud,Indonesia,Bali
2,2021-11-13,Astagina Resort Villa and Spa,8.5,21.0,2327.0,4.0,Seminyak,Indonesia,Bali
3,2021-11-13,Mulia Resort,8.9,161.0,372.0,5.0,Nusa Dua,Indonesia,Bali
4,2021-11-13,Mahanidhi Luxury Private Villa 4,NaN,39.0,NaN,4.0,Ubud,Indonesia,Bali


#### Step 4 - Export transformed data

In [ ]:
df.to_csv("C:\Users\agnes\switchdrive\SyncVM\CIP H21\Project\Stage Data\agoda_stage.csv", index=False)  

### SECTION 3: Load data

#### Step 1 - read packages needed

In [1]:
import mysql.connector as db
from mysql.connector import Error

#### Step 2 - Connect to MariaDB CIP database

In [2]:
connection = db.connect(
    user="admin",
    password="Cip2021!",
    host="localhost",
    database="CIP")

cursor = connection.cursor()

**Step 3 - Create tables to store data**

Create table to store source data

In [21]:
try:
    cursor.execute("CREATE TABLE `agoda_source` (\
  `name` varchar(100) NOT NULL,\
  `price` varchar(50),\
  `rating` varchar(50) ,\
  `number_of_reviews` varchar(50) ,\
  `address` varchar(100) ,\
  `stars` varchar(50) ,\
  `date` varchar(19)\
)")
    print("Table is created....")
except Error as e:
    print(e)

Table is created....


Create table to store stage data

In [22]:
try:
    cursor.execute("CREATE TABLE `agoda_stage` (\
  `date` varchar(50) NOT NULL,\
  `title` varchar(100),\
  `rate` double ,\
  `price` double ,\
  `review` double ,\
  `star` double ,\
  `city` varchar(100) ,\
  `country` varchar(100) ,\
  `region` varchar(50)\
)")
    print("Table is created....")
except Error as e:
    print(e)

Table is created....


#### Read the source file

In [23]:
df_source = pd.read_csv("../Project/Source Data/agoda_source/agoda_source.csv",sep=',')
source_data = df_source.astype(object).where(pd.notnull(df_source), None)

#### Read the stage file

In [24]:
df_stage= pd.read_csv("../Project/Stage Data/Agoda_stage.csv",sep=',')
stage_data = df_stage.astype(object).where(pd.notnull(df_stage), None)

#### Stpe 4 - Load data to the table

Load source data 

In [25]:
for i, row in source_data.iterrows():
    try: 
        sql = "INSERT INTO agoda_source (name, price, rating, number_of_reviews, address, stars, date)\
               VALUES(%s, %s, %s, %s, %s, %s, %s)"
        cursor.execute(sql, tuple(row))
        print("Record inserted") 
        connection.commit()
    except Error as e:
        print ("Error while connecting to MySQL", e)

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record i

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record i

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record i

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record i

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record i

...

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record i

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record i

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record i

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record i

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record i

Load stage data

In [26]:
for i, row in stage_data.iterrows():
    try: 
        sql = "INSERT INTO agoda_stage (date, title, rate, price, review, star, city, country, region)\
               VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s)"
        cursor.execute(sql, tuple(row))
        print("Record inserted") 
        connection.commit()
    except Error as e:
        print ("Error while connecting to MySQL", e)

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record i

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record i

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record i

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record i

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record i

...

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record i

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record i

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record i

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record i

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record i

#### Check the number of rows of the table

check agoda_source table

In [29]:
sql = "SELECT COUNT(*) FROM agoda_source"
cursor.execute(sql)

number_of_rows = cursor.fetchall()
print(number_of_rows)

[(16137,)]


check agoda_stage table

In [28]:
sql = "SELECT COUNT(*) FROM agoda_stage"
cursor.execute(sql)

number_of_rows_1 = cursor.fetchall()
print(number_of_rows_1)

[(13318,)]


#### Check the first 10 rows of the table

check agoda_source table

In [12]:
sql = "SELECT *FROM agoda_source LIMIT 10;"
cursor.execute(sql)
print(cursor.fetchall())

[('Amnaya Resort Kuta', '21', '9.2', '8,352 reviews', 'Kuta, Bali - 3.4 km to center', '4 star', '13-11-2021'), ('The Kayon Jungle Resort', '117', '9.4', '1,162 reviews', 'Ubud, Bali - 30.1 km to center', '5 star', '13-11-2021'), ('Astagina Resort Villa and Spa', '21', '8.5', '2,327 reviews', 'Seminyak, Bali - 99 m to center', '4 star', '13-11-2021'), ('Mulia Resort', '161', '8.9', '372 reviews', 'Nusa Dua, Bali - 13.8 km to center', '5 star', '13-11-2021'), ('Mahanidhi Luxury Private Villa #4', '39', 'N.A', 'N.A', 'Ubud, Bali - 22.2 km to center', '4 star', '13-11-2021'), ('Aksari Resort Ubud', '46', '9.0', '445 reviews', 'Ubud, Bali - 30.7 km to center', '4 star', '13-11-2021'), ('Aston Canggu Beach Resort', '25', '8.4', '371 reviews', 'Canggu, Bali - 2.7 km to center', '4 star', '13-11-2021'), ('IZE Seminyak', '41', '8.6', '2,463 reviews', 'Seminyak, Bali - City center', '4 star', '13-11-2021'), ('Escotel Semat', 'N.A', '10.0', '2 reviews', 'Canggu, Bali - 2.6 km to center', ' star'

check agoda_stage table

In [30]:
sql = "SELECT *FROM agoda_stage LIMIT 10;"
cursor.execute(sql)
print(cursor.fetchall())

[('2021-11-13', 'Amnaya Resort Kuta', 9.2, 21.0, 8352.0, 4.0, 'Kuta', 'Indonesia', 'Bali'), ('2021-11-13', 'The Kayon Jungle Resort', 9.4, 117.0, 1162.0, 5.0, 'Ubud', 'Indonesia', 'Bali'), ('2021-11-13', 'Astagina Resort Villa and Spa', 8.5, 21.0, 2327.0, 4.0, 'Seminyak', 'Indonesia', 'Bali'), ('2021-11-13', 'Mulia Resort', 8.9, 161.0, 372.0, 5.0, 'Nusa Dua', 'Indonesia', 'Bali'), ('2021-11-13', 'Mahanidhi Luxury Private Villa 4', None, 39.0, None, 4.0, 'Ubud', 'Indonesia', 'Bali'), ('2021-11-13', 'Aksari Resort Ubud', 9.0, 46.0, 445.0, 4.0, 'Ubud', 'Indonesia', 'Bali'), ('2021-11-13', 'Aston Canggu Beach Resort', 8.4, 25.0, 371.0, 4.0, 'Canggu', 'Indonesia', 'Bali'), ('2021-11-13', 'IZE Seminyak', 8.6, 41.0, 2463.0, 4.0, 'Seminyak', 'Indonesia', 'Bali'), ('2021-11-13', 'TheDarling Ocean View Villa UluwatuUngasan Bali', 9.1, 178.0, 11.0, 4.5, 'Uluwatu', 'Indonesia', 'Bali'), ('2021-11-13', 'Munduk Heaven Luxury Villas', 9.7, 88.0, 5.0, 0.0, 'Munduk', 'Indonesia', 'Bali')]
